# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Remove this
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
import numpy as np
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key



In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.sample(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
138,138,Hawaiian Paradise Park,19.5933,-154.9731,25.49,77,Clouds,6.69,US,1704415847
492,492,Kissidougou,9.1848,-10.0999,21.66,26,Clouds,0.84,GN,1704415879
483,483,Prince Rupert,54.3161,-130.3201,6.83,85,Rain,10.80,CA,1704415811
228,228,Xifeng,42.7372,124.7222,-11.58,87,Clear,1.83,CN,1704415854
472,472,Tura,25.5198,90.2201,12.56,72,Clear,1.45,IN,1704415877
530,530,Mogocha,53.7333,119.7667,-24.61,97,Clouds,0.60,RU,1704415885
37,37,Ilulissat,69.2167,-51.1000,-6.99,86,Snow,5.66,GL,1704415841
451,451,Chabahar,25.2919,60.6430,21.76,62,Clear,2.92,IR,1704415875
96,96,Lompoc,34.6391,-120.4579,16.08,64,Clear,7.20,US,1704415658
36,36,Ingeniero Jacobacci,-41.3427,-69.5448,20.62,30,Clear,8.22,AR,1704415757


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# %%capture --no-display

# configure the map_plot_1
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 1,
    frame_width = 700,
    frame_height = 500,
    color = "City",
    alpha = 0.5,
)

# Display the map plot_1
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

my_cities = city_data_df.loc[(city_data_df["Max Temp"] > 24) & (city_data_df["Max Temp"] < 27)]
my_cities = my_cities.dropna()
# Display sample data

my_cities.sample

<bound method NDFrame.sample of      City_ID            City      Lat       Lng  Max Temp  Humidity  \
5          5        Holualoa  19.6228 -155.9522     24.91        71   
10        10    Puerto Ayora  -0.7393  -90.3518     25.31        95   
28        28  Saint-Philippe -21.3585   55.7679     25.77        87   
32        32       Kilindoni  -7.9139   39.6668     26.71        81   
35        35     Salinópolis  -0.6136  -47.3561     26.15        89   
..       ...             ...      ...       ...       ...       ...   
573      573     Kailua-Kona  19.6406 -155.9956     26.98        70   
575      575     Lhokseumawe   5.1801   97.1507     25.81        81   
578      578      Mozambique -15.0342   40.7358     25.72        89   
582      582           Gamay  12.3854  125.2979     26.68        91   
583      583           Gurun   5.8172  100.4738     24.90        83   

    Cloudiness  Wind Speed Country        Date  
5        Clear        3.09      US  1704415838  
10      Clouds   

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = my_cities[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,Holualoa,US,19.6228,-155.9522,71,
10,Puerto Ayora,EC,-0.7393,-90.3518,95,
28,Saint-Philippe,RE,-21.3585,55.7679,87,
32,Kilindoni,TZ,-7.9139,39.6668,81,
35,Salinópolis,BR,-0.6136,-47.3561,89,
...,...,...,...,...,...,...
573,Kailua-Kona,US,19.6406,-155.9956,70,
575,Lhokseumawe,ID,5.1801,97.1507,81,
578,Mozambique,MZ,-15.0342,40.7358,89,
582,Gamay,PH,12.3854,125.2979,91,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")


# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
   
    latitude = row["Lat"]
    longitude = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Holualoa - nearest hotel: Kona Hotel
Puerto Ayora - nearest hotel: Hostal La Mirada De Solitario George
Saint-Philippe - nearest hotel: Le Baril
Kilindoni - nearest hotel: Ibizza Inn
Salinópolis - nearest hotel: Hotel Salinópolis
Harper - nearest hotel: No hotel found
Kapit - nearest hotel: Haip
Goléré - nearest hotel: Espace Aduna'm
Lázaro Cárdenas - nearest hotel: Hotel Sol del Pacífico
Myeik - nearest hotel: Green Eyes Hotel
Nova Viçosa - nearest hotel: Pousada Pontal da Barra
Puerto Baquerizo Moreno - nearest hotel: Dory's House
Hawaiian Paradise Park - nearest hotel: No hotel found
Dampit - nearest hotel: No hotel found
Barra de Santo Antônio - nearest hotel: Resort Fazenda Fiore
Bucerías - nearest hotel: Royal Decameron Complex
Utrik - nearest hotel: No hotel found
Tôlanaro - nearest hotel: Hôtel Mahavokey
Vung Tau - nearest hotel: No hotel found
Ixtapa - nearest hotel: Ma. Cristina
São João da Barra - nearest hotel: Pousada Mediterrâneo
Lobito - nearest hot

,City,Country,Lat,Lng,Humidity,Hotel Name
5,Holualoa,US,19.6228,-155.9522,71,Kona Hotel
10,Puerto Ayora,EC,-0.7393,-90.3518,95,Hostal La Mirada De Solitario George
28,Saint-Philippe,RE,-21.3585,55.7679,87,Le Baril
32,Kilindoni,TZ,-7.9139,39.6668,81,Ibizza Inn
35,Salinópolis,BR,-0.6136,-47.3561,89,Hotel Salinópolis
...,...,...,...,...,...,...
573,Kailua-Kona,US,19.6406,-155.9956,70,Kona Seaside Hotel
575,Lhokseumawe,ID,5.1801,97.1507,81,Vina Vira Hotel
578,Mozambique,MZ,-15.0342,40.7358,89,Ruby Backpackers
582,Gamay,PH,12.3854,125.2979,91,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols=["Country","Hotel Name"],
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 1,
    frame_width = 700,
    frame_height = 500,
    color = "City",
    alpha = 0.5,
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)